In [1]:
! pip install datasets transformers[sentencepiece]


[notice] A new release of pip available: 22.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset

# load_dataset returns a python object that you can index
dataset = load_dataset("swiss_judgment_prediction", "all", split="train")
dataset[0]

{'id': 2,
 'year': 2000,
 'text': 'A.- Der 1955 geborene V._ war seit 1. September 1986 hauptberuflich als technischer Kaufmann bei der Firma A._ AG tätig und im Rahmen einer Nebenbeschäftigung (Nachtarbeit) ab Mai 1990 bei einem Bewachungsdienst angestellt gewesen, als er am 10. Februar 1991 in Norwegen beim Hundeschlittenfahren eine Muskelruptur im Bereich des linken Oberschenkels erlitt. Die Verletzung wurde am 26. Februar 1991 mittels Muskelnaht operativ versorgt (Bericht des Dr. med. B._, Oberarzt, Chirurgische Klinik X._ vom 28. Februar 1991). Beweglichkeits- und Sensibilitätsausfälle führten zum Beizug des Dr. med. W._, Spezialarzt FMH Neurologie, welcher eine Ischiadicusparese links, wahrscheinlich traumatisch bedingt, diagnostizierte (Bericht vom 5. März 1991). Dr. med. S._, Spezialarzt für Chirurgie FMH, Chefarzt Spital X._ (Bericht vom 28. Oktober 1992) bestätigte, dass es bei der Operation vom 10. Februar 1991 zu einer Druckschädigung des Ischiasnerv gekommen sei. Bei Verso

In [3]:
# Convert the output format to pandas.DataFrame
dataset.set_format("pandas")
dataset[0]

,id,year,text,label,language,region,canton,legal area,source_language
0,2,2000,A.- Der 1955 geborene V._ war seit 1. Septembe...,0,de,Zürich,zh,insurance law,n/a


In [4]:
dataset.__getitem__(0)

,id,year,text,label,language,region,canton,legal area,source_language
0,2,2000,A.- Der 1955 geborene V._ war seit 1. Septembe...,0,de,Zürich,zh,insurance law,n/a


In [5]:
df = dataset.to_pandas()
# See top 5 results
df.head()

,id,year,text,label,language,region,canton,legal area,source_language
0,2,2000,A.- Der 1955 geborene V._ war seit 1. Septembe...,0,de,Zürich,zh,insurance law,n/a
1,3,2000,"Ansprüche nach OHG, hat sich ergeben: A.- X._ ...",1,de,Central Switzerland,lu,public law,n/a
2,4,2000,Art. 4 aBV (Strafverfahren wegen falschen Zeug...,0,de,Northwestern Switzerland,ag,public law,n/a
3,5,2000,"Art. 5 Ziff. 1 EMRK (Haftentlassung), hat sich...",1,de,n/a,n/a,public law,n/a
4,6,2000,"Mietvertrag, hat sich ergeben: A.- Die CT Cond...",0,de,n/a,n/a,civil law,n/a


In [6]:
# How are languages distributed across regions?
df.groupby("region")["language"].value_counts()

region                    language
Central Switzerland       de           4778
                          it              1
Eastern Switzerland       de           5650
                          it             57
Espace Mittelland         de           5150
                          fr           3104
                          it              3
Federation                de           1011
                          fr            227
                          it             70
Northwestern Switzerland  de           5654
                          fr              1
Région lémanique          fr          13100
                          de            336
Ticino                    it           2249
                          de              6
Zürich                    de           8785
                          fr              3
n/a                       fr           4744
                          de           4088
                          it            692
Name: language, dtype: int64

In [7]:
# Which legal area is most common?
df["legal area"].value_counts()

public law       15173
penal law        11795
civil law        11477
insurance law    11142
social law        9727
other              395
Name: legal area, dtype: int64

In [8]:
from transformers import AutoTokenizer

# Load a pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# Tokenize the `text` column
dataset.map(lambda x : tokenizer(x["text"]))
# Error is expected here! This is because we forgot to reset the dataset to be in "Arrow Format"

Map:   0%|          | 0/59709 [00:00<?, ? examples/s]

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
# Reset back to Arrow format
dataset.reset_format()
# Now we can tokenize!
dataset.map(lambda x : tokenizer(x["text"]))